Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [18]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from sklearn.linear_model import LogisticRegression

First reload the data we generated in `1_notmnist.ipynb`.

In [19]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

train_labels_orig = train_labels
valid_labels_orig = valid_labels
test_labels_orig = test_labels

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [20]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [21]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Regularization For Logistic Model

In [22]:
samples=100000
C=1 #1e42

lr = LogisticRegression(C=C)
nsamples, img_shape = train_dataset.shape
d2_train_dataset = train_dataset.reshape((nsamples,img_shape))[:samples]

nsamples, img_shape = test_dataset.shape
d2_test_dataset = test_dataset.reshape((nsamples,img_shape))[:samples]


print(train_labels[:samples].shape)
lr.fit(d2_train_dataset,train_labels_orig[:samples])
y_pred = lr.predict(d2_test_dataset)

from sklearn import metrics
print(metrics.accuracy_score(y_pred, test_labels_orig[:samples]))
print(metrics.f1_score(y_pred, test_labels_orig[:samples], average='micro'))

(100000, 10)
0.8907
0.8907


### L2 Regularization For Neural Net without hidden layer

In [23]:
train_subset = 10000
beta=0.07

graph = tf.Graph()
with graph.as_default():
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    logits = tf.matmul(train_dataset, weights) + biases
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=train_labels, logits=logits))
    # Test accuracy: 83.9% w/o regularization
    loss=loss+tf.multiply(beta, tf.nn.l2_loss(weights))

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
    tf.matmul(valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_dataset, weights) + biases)

In [24]:
num_steps = 801

from datetime import datetime

start = datetime.now()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

tot = str(datetime.now()-start)
print("Total Time={}".format(tot))

Initialized
Loss at step 0: 233.981430
Validation accuracy: 10.6%
Loss at step 100: 1.082748
Validation accuracy: 81.1%
Loss at step 200: 0.931287
Validation accuracy: 81.0%
Loss at step 300: 0.930516
Validation accuracy: 81.0%
Loss at step 400: 0.930312
Validation accuracy: 81.0%
Loss at step 500: 0.930240
Validation accuracy: 81.0%
Loss at step 600: 0.930207
Validation accuracy: 81.0%
Loss at step 700: 0.930197
Validation accuracy: 81.0%
Loss at step 800: 0.930194
Validation accuracy: 81.0%
Test accuracy: 87.9%
Total Time=0:26:58.554839


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In the overfitting, the train data will show high accuracy in compare to validation and the test.

In [25]:
def network(x, drop=False):
    l1 = tf.add(tf.matmul(x,  w1), b1)
    l1 = tf.nn.relu(l1)
    if drop:
        l1 = tf.nn.dropout(l1, 0.5)
    outer = tf.add(tf.matmul(l1, w_outer), bias_outer)
    
    return outer

In [26]:
train_subset = 5000
hidden_layers_size = 500
beta=0.07

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    w_outer = tf.Variable(tf.truncated_normal((hidden_layers_size, num_labels)))

    b1 = tf.Variable(tf.zeros([hidden_layers_size])),
    bias_outer = tf.Variable(tf.zeros([num_labels]))
    
    logits = network(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    

    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction =  tf.nn.softmax(network(tf_valid_dataset))
    test_prediction = tf.nn.softmax(network(tf_test_dataset))

In [27]:
num_steps=801

from datetime import datetime

start = datetime.now()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

tot = str(datetime.now()-start)
print("Total Time={}".format(tot))

Initialized
Loss at step 0: 284.583069
Training accuracy: 5.7%
Validation accuracy: 19.2%
Loss at step 100: 10.527025
Training accuracy: 81.4%
Validation accuracy: 73.2%
Loss at step 200: 4.360045
Training accuracy: 87.9%
Validation accuracy: 73.0%
Loss at step 300: 1.364158
Training accuracy: 94.6%
Validation accuracy: 72.9%
Loss at step 400: 0.390220
Training accuracy: 97.2%
Validation accuracy: 73.2%
Loss at step 500: 0.089656
Training accuracy: 99.1%
Validation accuracy: 73.1%
Loss at step 600: 0.024793
Training accuracy: 99.8%
Validation accuracy: 73.0%
Loss at step 700: 0.012456
Training accuracy: 99.8%
Validation accuracy: 73.0%
Loss at step 800: 0.006092
Training accuracy: 99.8%
Validation accuracy: 73.1%
Test accuracy: 81.7%
Total Time=0:08:14.957490


## -------------------------

### With the mini batches

In [28]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    w_outer = tf.Variable(tf.truncated_normal((hidden_layers_size, num_labels)))

    b1 = tf.Variable(tf.zeros([hidden_layers_size])),
    bias_outer = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = network(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = network(tf_valid_dataset)
    test_prediction = network(tf_test_dataset)

In [29]:
num_steps = 3001

from datetime import datetime

start = datetime.now()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

tot = str(datetime.now()-start)
print("Total Time={}".format(tot))

Initialized
Minibatch loss at step 0: 246.792252
Minibatch accuracy: 7.8%
Validation accuracy: 20.1%
Minibatch loss at step 500: 9.072532
Minibatch accuracy: 85.2%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 10.204161
Minibatch accuracy: 77.3%
Validation accuracy: 78.3%
Minibatch loss at step 1500: 3.586279
Minibatch accuracy: 89.8%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 3.753148
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 3.479273
Minibatch accuracy: 83.6%
Validation accuracy: 82.1%
Minibatch loss at step 3000: 2.818547
Minibatch accuracy: 80.5%
Validation accuracy: 81.7%
Test accuracy: 88.9%
Total Time=0:01:49.846983


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [30]:
train_subset = 5000
hidden_layers_size = 500
beta=0.07

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    w_outer = tf.Variable(tf.truncated_normal((hidden_layers_size, num_labels)))

    b1 = tf.Variable(tf.zeros([hidden_layers_size])),
    bias_outer = tf.Variable(tf.zeros([num_labels]))
    
    logits = network(tf_train_dataset, True)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    

    optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction =  tf.nn.softmax(network(tf_valid_dataset, True))
    test_prediction = tf.nn.softmax(network(tf_test_dataset, True))

In [31]:
num_steps=801

from datetime import datetime

start = datetime.now()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

tot = str(datetime.now()-start)
print("Total Time={}".format(tot))

Initialized
Loss at step 0: 322.726685
Training accuracy: 11.9%
Validation accuracy: 18.2%
Loss at step 100: 32.135086
Training accuracy: 74.9%
Validation accuracy: 72.2%
Loss at step 200: 19.821112
Training accuracy: 78.4%
Validation accuracy: 72.9%
Loss at step 300: 13.313540
Training accuracy: 81.0%
Validation accuracy: 73.9%
Loss at step 400: 10.298458
Training accuracy: 82.3%
Validation accuracy: 74.5%
Loss at step 500: 7.137795
Training accuracy: 83.8%
Validation accuracy: 74.8%
Loss at step 600: 6.406398
Training accuracy: 84.6%
Validation accuracy: 74.8%
Loss at step 700: 5.193758
Training accuracy: 85.3%
Validation accuracy: 75.2%
Loss at step 800: 4.580975
Training accuracy: 85.4%
Validation accuracy: 75.3%
Test accuracy: 82.6%
Total Time=0:09:43.617356


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


Things can be done in addition to above:
* Increase the train data subset
* Increase number of steps
* Try different activation functions
* Dropout keep_prob 

In [32]:
keep_prob=0.8
def network_for_perf(x, drop=False):
    l1 = tf.add(tf.matmul(x,  w1), b1)
    l1 = tf.nn.relu(l1)
    #if drop:
    l1 = tf.nn.dropout(l1, keep_prob)
    
    l2 = tf.add(tf.matmul(l1,  w2), b2)
    l2 = tf.nn.relu(l2)
    l2 = tf.nn.dropout(l2, keep_prob)
    
    l3 = tf.add(tf.matmul(l2,  w3), b3)
    l3 = tf.nn.relu(l3)
#     l3 = tf.nn.dropout(l3, keep_prob)
    outer = tf.add(tf.matmul(l2, w_outer), bias_outer)
    
    return outer

In [33]:
batch_size = 512
hidden_layers_size = 512
learn_rate=0.001

# keep_prob=0.8
# def network(x, drop=False):
#     l1 = tf.add(tf.matmul(x,  w1), b1)
#     l1 = tf.nn.relu(l1)
#     #if drop:
#     l1 = tf.nn.dropout(l1, keep_prob)
    
#     l2 = tf.add(tf.matmul(l1,  w2), b2)
#     l2 = tf.nn.relu(l2)
#     l2 = tf.nn.dropout(l2, keep_prob)
    
#     l3 = tf.add(tf.matmul(l2,  w3), b3)
#     l3 = tf.nn.relu(l3)
# #     l3 = tf.nn.dropout(l3, keep_prob)
#     outer = tf.add(tf.matmul(l2, w_outer), bias_outer)
    
#     return outer


graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layers_size]))
    w2 = tf.Variable(tf.truncated_normal([hidden_layers_size, hidden_layers_size]))
    w3 = tf.Variable(tf.truncated_normal([hidden_layers_size, hidden_layers_size]))
    w_outer = tf.Variable(tf.truncated_normal((hidden_layers_size, num_labels)))

    b1 = tf.Variable(tf.zeros([hidden_layers_size]))
    b2 = tf.Variable(tf.zeros([hidden_layers_size]))
    b3 = tf.Variable(tf.zeros([hidden_layers_size]))
    bias_outer = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = network(tf_train_dataset)
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    #optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
    optimizer = tf.train.RMSPropOptimizer(learn_rate).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = network(tf_valid_dataset)
    test_prediction = network(tf_test_dataset)

In [34]:
num_steps = 300000

from datetime import datetime

start = datetime.now()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

tot = str(datetime.now()-start)
print("Total Time={}".format(tot))

Initialized
Minibatch loss at step 0: 225.862457
Minibatch accuracy: 14.1%
Validation accuracy: 13.2%
Minibatch loss at step 500: 14.402805
Minibatch accuracy: 78.9%
Validation accuracy: 79.0%
Minibatch loss at step 1000: 10.270929
Minibatch accuracy: 82.0%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 9.312311
Minibatch accuracy: 84.2%
Validation accuracy: 81.9%
Minibatch loss at step 2000: 9.883364
Minibatch accuracy: 80.3%
Validation accuracy: 82.3%
Minibatch loss at step 2500: 7.506279
Minibatch accuracy: 84.0%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 5.894453
Minibatch accuracy: 86.5%
Validation accuracy: 83.2%
Minibatch loss at step 3500: 4.691394
Minibatch accuracy: 86.7%
Validation accuracy: 83.5%
Minibatch loss at step 4000: 3.955575
Minibatch accuracy: 88.3%
Validation accuracy: 84.0%
Minibatch loss at step 4500: 3.161830
Minibatch accuracy: 87.1%
Validation accuracy: 84.1%
Minibatch loss at step 5000: 4.413912
Minibatch accuracy: 85.7%
Validation acc

Validation accuracy: 86.4%
Minibatch loss at step 45000: 0.029515
Minibatch accuracy: 99.0%
Validation accuracy: 86.6%
Minibatch loss at step 45500: 0.133680
Minibatch accuracy: 97.1%
Validation accuracy: 86.6%
Minibatch loss at step 46000: 0.059373
Minibatch accuracy: 98.6%
Validation accuracy: 86.1%
Minibatch loss at step 46500: 0.110743
Minibatch accuracy: 98.0%
Validation accuracy: 86.5%
Minibatch loss at step 47000: 0.100048
Minibatch accuracy: 98.6%
Validation accuracy: 86.2%
Minibatch loss at step 47500: 0.213561
Minibatch accuracy: 98.0%
Validation accuracy: 86.7%
Minibatch loss at step 48000: 0.091977
Minibatch accuracy: 98.4%
Validation accuracy: 86.3%
Minibatch loss at step 48500: 0.081661
Minibatch accuracy: 98.6%
Validation accuracy: 86.2%
Minibatch loss at step 49000: 0.122085
Minibatch accuracy: 98.6%
Validation accuracy: 86.8%
Minibatch loss at step 49500: 0.182003
Minibatch accuracy: 98.0%
Validation accuracy: 86.8%
Minibatch loss at step 50000: 0.046138
Minibatch accu

Minibatch loss at step 89500: 0.075609
Minibatch accuracy: 98.8%
Validation accuracy: 87.2%
Minibatch loss at step 90000: 0.184356
Minibatch accuracy: 98.4%
Validation accuracy: 86.7%
Minibatch loss at step 90500: 0.044278
Minibatch accuracy: 99.2%
Validation accuracy: 87.1%
Minibatch loss at step 91000: 0.017401
Minibatch accuracy: 99.6%
Validation accuracy: 87.3%
Minibatch loss at step 91500: 0.194648
Minibatch accuracy: 98.6%
Validation accuracy: 86.7%
Minibatch loss at step 92000: 0.053491
Minibatch accuracy: 99.2%
Validation accuracy: 87.1%
Minibatch loss at step 92500: 0.255068
Minibatch accuracy: 98.4%
Validation accuracy: 86.9%
Minibatch loss at step 93000: 0.051609
Minibatch accuracy: 99.0%
Validation accuracy: 87.2%
Minibatch loss at step 93500: 0.013404
Minibatch accuracy: 99.4%
Validation accuracy: 87.2%
Minibatch loss at step 94000: 0.153836
Minibatch accuracy: 98.6%
Validation accuracy: 87.1%
Minibatch loss at step 94500: 0.035006
Minibatch accuracy: 99.2%
Validation accu

Validation accuracy: 87.3%
Minibatch loss at step 134000: 0.057804
Minibatch accuracy: 98.8%
Validation accuracy: 86.9%
Minibatch loss at step 134500: 0.069904
Minibatch accuracy: 98.4%
Validation accuracy: 86.8%
Minibatch loss at step 135000: 0.153428
Minibatch accuracy: 98.6%
Validation accuracy: 86.4%
Minibatch loss at step 135500: 0.145979
Minibatch accuracy: 99.0%
Validation accuracy: 87.1%
Minibatch loss at step 136000: 0.045337
Minibatch accuracy: 99.2%
Validation accuracy: 87.1%
Minibatch loss at step 136500: 0.330663
Minibatch accuracy: 98.6%
Validation accuracy: 87.2%
Minibatch loss at step 137000: 0.446120
Minibatch accuracy: 96.5%
Validation accuracy: 87.0%
Minibatch loss at step 137500: 0.121217
Minibatch accuracy: 98.6%
Validation accuracy: 87.1%
Minibatch loss at step 138000: 0.154385
Minibatch accuracy: 99.2%
Validation accuracy: 87.3%
Minibatch loss at step 138500: 0.055644
Minibatch accuracy: 99.2%
Validation accuracy: 87.1%
Minibatch loss at step 139000: 0.017523
Min

Minibatch loss at step 178000: 0.165580
Minibatch accuracy: 99.0%
Validation accuracy: 87.3%
Minibatch loss at step 178500: 0.191682
Minibatch accuracy: 99.2%
Validation accuracy: 87.2%
Minibatch loss at step 179000: 0.167319
Minibatch accuracy: 99.0%
Validation accuracy: 87.3%
Minibatch loss at step 179500: 0.010885
Minibatch accuracy: 99.6%
Validation accuracy: 87.3%
Minibatch loss at step 180000: 0.054305
Minibatch accuracy: 99.6%
Validation accuracy: 87.5%
Minibatch loss at step 180500: 0.008995
Minibatch accuracy: 99.4%
Validation accuracy: 87.2%
Minibatch loss at step 181000: 0.038382
Minibatch accuracy: 99.4%
Validation accuracy: 87.4%
Minibatch loss at step 181500: 0.242934
Minibatch accuracy: 99.0%
Validation accuracy: 87.3%
Minibatch loss at step 182000: 0.079949
Minibatch accuracy: 99.2%
Validation accuracy: 87.2%
Minibatch loss at step 182500: 0.593189
Minibatch accuracy: 97.9%
Validation accuracy: 87.3%
Minibatch loss at step 183000: 0.123170
Minibatch accuracy: 98.6%
Vali

Validation accuracy: 87.3%
Minibatch loss at step 222500: 0.017186
Minibatch accuracy: 99.6%
Validation accuracy: 87.2%
Minibatch loss at step 223000: 0.043914
Minibatch accuracy: 99.2%
Validation accuracy: 87.4%
Minibatch loss at step 223500: 0.085754
Minibatch accuracy: 99.2%
Validation accuracy: 87.3%
Minibatch loss at step 224000: 0.092820
Minibatch accuracy: 99.0%
Validation accuracy: 87.4%
Minibatch loss at step 224500: 0.043600
Minibatch accuracy: 99.2%
Validation accuracy: 87.5%
Minibatch loss at step 225000: 0.088078
Minibatch accuracy: 98.8%
Validation accuracy: 87.4%
Minibatch loss at step 225500: 0.003360
Minibatch accuracy: 99.8%
Validation accuracy: 87.5%
Minibatch loss at step 226000: 0.110846
Minibatch accuracy: 99.2%
Validation accuracy: 87.4%
Minibatch loss at step 226500: 0.102320
Minibatch accuracy: 98.8%
Validation accuracy: 87.3%
Minibatch loss at step 227000: 0.041802
Minibatch accuracy: 99.2%
Validation accuracy: 87.5%
Minibatch loss at step 227500: 0.074250
Min

Minibatch loss at step 266500: 0.179574
Minibatch accuracy: 99.2%
Validation accuracy: 87.5%
Minibatch loss at step 267000: 0.005132
Minibatch accuracy: 99.8%
Validation accuracy: 87.6%
Minibatch loss at step 267500: 0.015259
Minibatch accuracy: 99.4%
Validation accuracy: 87.8%
Minibatch loss at step 268000: 0.398993
Minibatch accuracy: 97.9%
Validation accuracy: 87.6%
Minibatch loss at step 268500: 0.037991
Minibatch accuracy: 99.6%
Validation accuracy: 87.5%
Minibatch loss at step 269000: 0.289748
Minibatch accuracy: 98.4%
Validation accuracy: 87.6%
Minibatch loss at step 269500: 0.026957
Minibatch accuracy: 99.6%
Validation accuracy: 87.5%
Minibatch loss at step 270000: 0.011142
Minibatch accuracy: 99.8%
Validation accuracy: 87.8%
Minibatch loss at step 270500: 0.324073
Minibatch accuracy: 97.9%
Validation accuracy: 87.2%
Minibatch loss at step 271000: 0.003342
Minibatch accuracy: 99.8%
Validation accuracy: 87.8%
Minibatch loss at step 271500: 0.006886
Minibatch accuracy: 99.4%
Vali